# Simple Convolutional Neural Network classification demo

Using the MNIST dataset. For the example using gpu's see: https://nextjournal.com/sdanisch/julia-gpu-programming

If you haven't done it yet, you'll need to install the packages below.

In [ ]:
#using Pkg
#Pkg.add(Flux)
#etc., 

#or install using repo mode:

#] add Flux
#] add Flux ImageMagick
#] add Flux QuartzImageIO
#] add FileIO
#] add ImageShow

In [ ]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition

In [ ]:
# Read in MNIST digits, set up training set

imgs = MNIST.images()

labels = onehotbatch(MNIST.labels(), 0:9)

# Partition into batches of size 1,000
train = [(cat(float.(imgs[i])..., dims = 4), labels[:,i])
         for i in partition(1:60_000, 1000)]

In [ ]:
# Prepare test set (first 1,000 images)
tX = cat(float.(MNIST.images(:test)[1:1000])..., dims = 4)
tY = onehotbatch(MNIST.labels(:test)[1:1000], 0:9)

Set up network architecture, define the loss function

In [5]:
m = Chain(
  Conv((2,2), 1=>16, relu),
  x -> maxpool(x, (2,2)),
  Conv((2,2), 16=>8, relu),
  x -> maxpool(x, (2,2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(288, 10), softmax)

#m(train[1][1])

loss(x, y) = crossentropy(m(x), y)

accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

evalcb = throttle(() -> @show(accuracy(tX, tY)), 10)
opt = ADAM(Flux.params(m));

In [6]:
#To keep it quick, run only 2 batches
# train
for i = 1:2
    Flux.train!(loss, train, opt, cb = evalcb)
end

accuracy(tX, tY) = 0.084
accuracy(tX, tY) = 0.199
accuracy(tX, tY) = 0.376
accuracy(tX, tY) = 0.563
accuracy(tX, tY) = 0.68
accuracy(tX, tY) = 0.719
accuracy(tX, tY) = 0.774
accuracy(tX, tY) = 0.799


Test an image

In [ ]:
using Colors, FileIO, ImageShow
N = 22
img = tX[:, :, 1:1, N:N]
println("Predicted: ", Flux.onecold(m(img)) .- 1)
Gray.(collect(tX[:, :, 1, N]))